In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model

# reflect the tables


In [ ]:
# View all of the classes that automap found


In [ ]:
# Save references to each table


In [ ]:
# Create our session (link) from Python to the DB


# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()

1.	Use the SQLAlchemy create_engine() function to connect to your SQLite database.
2.	Use the SQLAlchemy automap_base() function to reflect your tables into classes, and then save references to the classes named station and measurement.
3.	Link Python to the database by creating a SQLAlchemy session.
IMPORTANT: Remember to close your session at the end of your notebook.
4.	Perform a precipitation analysis and then a station analysis by completing the steps in the following two subsections.
5.	Precipitation Analysis - Find the most recent date in the dataset.
6.	Using that date, get the previous 12 months of precipitation data by querying the previous 12 months of data.
HINT: Select only the "date" and "prcp" values.
7.	Load the query results into a Pandas DataFrame, and set the index to the "date" column.
8.	Sort the DataFrame values by "date".
9.	Plot the results by using the DataFrame plot method, as the following image shows:
10.	Use Pandas to print the summary statistics for the precipitation data.
11.	Station Analysis Design a query to calculate the total number of stations in the dataset.
12.	Design a query to find the most-active stations (that is, the stations that have the most rows). To do so, complete the following steps:
HINT: List the stations and observation counts in descending order.
13.	Answer the following question: which station id has the greatest number of observations?
14.	Using the most-active station id, calculate the lowest, highest, and average temperatures.
15.	Design a query to get the previous 12 months of temperature observation (TOBS) data. To do so, complete the following steps:
a.	Filter by the station that has the greatest number of observations.
b.	Query the previous 12 months of TOBS data for that station.
c.	Plot the results as a histogram with bins=12, as the following image shows:
d.	A screenshot depicts the histogram. Close your session.
